In [ ]:
import base64
import json
import sys
import os
import github
from github import Github
import hashlib
from flask import Flask, redirect, request

In [ ]:
gh = Github(os.environ['GITHUB_TOKEN'])
user = gh.get_user()

In [ ]:
app = Flask("colaburl")

In [ ]:
@app.route("/", methods=['GET'])
def main():
  assert len({'py', 'py64', 'ipynb'} & request.args.keys()) == 1
  if 'ipynb' in request.args:
    content = request.args['ipynb']
  else:
    if 'py' in request.args:
      content = request.args['py']
    elif 'py64' in request.args:
      content = base64.b64decode(request.args['py64'])
    cells = content.split('\n\n')
    content = json.dumps({
      "nbformat": 4,
      "nbformat_minor": 0,
      "metadata": {},
      "cells": [
        {
          "cell_type": "code",
          "source": cell.split('\n'),
          "metadata": {"id": str(i)}
        } for i, cell in enumerate(cells)
      ]
    })
  name = request.args.get('name', 'notebook.ipynb')
  assert '/' not in name
  assert name.endswith('.ipynb')

In [ ]:
  id = hashlib.sha256(json.dumps({'name': name, 'content': content}).encode()).hexdigest()
  repo = None
  try:
    repo = user.create_repo(id)
  except github.GithubException as ex:
    if ex.status != 422:
      raise

In [ ]:
  if repo:
    repo.create_file(name, 'autocommit', content)

In [ ]:
  return redirect(f'https://colab.research.google.com/github/colaburl/{id}/blob/main/{name}')	